In [1]:
import vertexai
from vertexai.generative_models import GenerativeModel, SafetySetting

/usr/local/lib/python3.12/dist-packages/google/cloud/aiplatform/models.py:52: FutureWarning: Support for google-cloud-storage < 3.0.0 will be removed in a future version of google-cloud-aiplatform. Please upgrade to google-cloud-storage >= 3.0.0.
  from google.cloud.aiplatform.utils import gcs_utils


In [2]:
# Configure
PROJECT_ID = "qwiklabs-gcp-03-e39ec6df7e97"
LOCATION = "us-central1"
MODEL_NAME = "gemini-2.5-flash"

In [3]:
vertexai.init(project=PROJECT_ID, location=LOCATION)

In [17]:
%%writefile gemini_funcs.py
import vertexai
from vertexai.generative_models import GenerativeModel

# Initialize Vertex AI (Ensure this is run in the environment where this file is imported)
# Note: In a real app, you might handle init differently, but for Colab this works if
# vertexai.init() was called in the notebook beforehand or we can rely on default creds.

def classify_user_question(text: str) -> str:
    """
    Classifies input text into: Employment, General Information,
    Emergency Services, or Tax Related.
    """
    model = GenerativeModel("gemini-2.5-flash")

    prompt = f"""
    You are a government intake assistant. Classify the following text into exactly
    one of these categories: 'Employment', 'General Information', 'Emergency Services', 'Tax Related'.

    Do not explain. Return only the category name.

    Text: {text}
    Category:
    """

    response = model.generate_content(
        prompt,
        generation_config={"temperature": 0.0}
    )
    return response.text.strip()

def generate_social_post(topic: str, context: str) -> str:
    """
    Generates a social media post for government announcements.
    """
    model = GenerativeModel("gemini-2.5-flash")

    prompt = f"""
    Write a concise, engaging social media post (Twitter/X style) for a local government account.

    Topic: {topic}
    Details: {context}

    Requirements:
    - Tone: Official yet helpful
    - Include 2 relevant hashtags
    - Include 1 emoji suitable for the topic
    """

    response = model.generate_content(
        prompt,
        generation_config={"temperature": 0.7}
    )
    return response.text.strip()

Overwriting gemini_funcs.py


In [18]:
import pytest
from vertexai.generative_models import GenerativeModel

In [19]:
%%writefile test_gemini_funcs.py
import pytest
from gemini_funcs import classify_user_question, generate_social_post

# 1. Test Classification
def test_classify_employment():
    input_text = "I was recently fired and need to file for unemployment benefits."
    category = classify_user_question(input_text)
    assert category == "Employment"

def test_classify_tax():
    input_text = "Where do I submit my W-2 forms?"
    category = classify_user_question(input_text)
    assert category == "Tax Related"

# 2. Test Social Media Generation
def test_social_post_structure():
    topic = "Snow Emergency"
    context = "Parking ban in effect from 6 PM tonight until 6 AM tomorrow."
    post = generate_social_post(topic, context)

    # Assertions
    assert len(post) > 10
    assert "#" in post  # Check for hashtags
    assert isinstance(post, str)

Overwriting test_gemini_funcs.py


In [20]:
!pytest test_gemini_funcs.py -W ignore

============================= test session starts ==============================
platform linux -- Python 3.12.12, pytest-8.4.2, pluggy-1.6.0
rootdir: /content
plugins: langsmith-0.4.38, typeguard-4.4.4, anyio-4.11.0
collected 3 items                                                              

test_gemini_funcs.py ...                                                 [100%]

============================== 3 passed in 14.59s ==============================


In [31]:
import pandas as pd
from gemini_funcs import classify_user_question

# 1. Define dataset with Ground Truth
classification_data = pd.DataFrame([
    {
        "text": "I lost my job and need to apply for benefits.",
        "ground_truth": "Employment"
    },
    {
        "text": "When is the library open on Sundays?",
        "ground_truth": "General Information"
    },
    {
        "text": "My house is on fire!",
        "ground_truth": "Emergency Services"
    },
    {
        "text": "How do I deduct home office expenses?",
        "ground_truth": "Tax Related"
    }
])

print("Running classification...")
classification_data["prediction"] = classification_data["text"].apply(classify_user_question)

# 3. Evaluate (Exact Match)
classification_data["is_correct"] = classification_data["prediction"] == classification_data["ground_truth"]

# Calculate accuracy percentage
accuracy = classification_data["is_correct"].mean() * 100

print(f"Model Accuracy: {accuracy}%")

# Show incorrect rows if any
if accuracy < 100:
    print("\nIncorrect Predictions:")
    display(classification_data[classification_data["is_correct"] == False])
else:
    print("\nPerfect score!")
    display(classification_data)

Running classification...
Model Accuracy: 100.0%

Perfect score!


,text,ground_truth,prediction,is_correct
0,I lost my job and need to apply for benefits.,Employment,Employment,True
1,When is the library open on Sundays?,General Information,General Information,True
2,My house is on fire!,Emergency Services,Emergency Services,True
3,How do I deduct home office expenses?,Tax Related,Tax Related,True


In [34]:
import pandas as pd
from vertexai.evaluation import EvalTask, PointwiseMetric, MetricPromptTemplateExamples
from gemini_funcs import generate_social_post

# 1. Prepare Data
#    We create a 'prompt' column that represents what the model was asked to do.
#    This helps the Evaluator understand the instructions.
social_data = pd.DataFrame([
    {
        "topic": "Heat Wave",
        "context": "Temperatures reaching 105F. Cooling centers open at City Hall."
    },
    {
        "topic": "Road Closure",
        "context": "Main St closed for parade Saturday 9am-12pm."
    }
])

# 2. Run your function (Generate the 'response')
print("Generating social posts...")
social_data["response"] = social_data.apply(
    lambda row: generate_social_post(row["topic"], row["context"]), axis=1
)

# 3. Create the 'prompt' column for the Evaluator
#    The evaluator needs to know what the instruction was to judge if the response is good.
#    We combine topic and context to mimic the instruction.
social_data["prompt"] = social_data.apply(
    lambda row: f"Write a social post about {row['topic']}. Details: {row['context']}", axis=1
)

# 4. Initialize EvalTask with the Standardized DataFrame
#    The dataframe now has 'prompt' and 'response' columns, which EvalTask automatically detects.
eval_task = EvalTask(
    dataset=social_data,
    metrics=[
        PointwiseMetric(
            metric="fluency",
            metric_prompt_template=MetricPromptTemplateExamples.get_prompt_template("fluency")
        ),
        PointwiseMetric(
            metric="text_quality",
            metric_prompt_template=MetricPromptTemplateExamples.get_prompt_template("text_quality")
        )
    ],
    experiment="social-post-eval-v2"
)

# 5. Run Evaluation
results = eval_task.evaluate()

# Display Results
display(pd.DataFrame([results.summary_metrics]))
display(results.metrics_table)

Generating social posts...


INFO:vertexai.evaluation._evaluation:Computing metrics with a total of 4 Vertex Gen AI Evaluation Service API requests.
100%|██████████| 4/4 [00:13<00:00,  3.35s/it]
INFO:vertexai.evaluation._evaluation:All 4 metric requests are successfully computed.
INFO:vertexai.evaluation._evaluation:Evaluation Took:13.403622186000575 seconds


,row_count,fluency/mean,fluency/std,text_quality/mean,text_quality/std
0,2,5.0,0.0,5.0,0.0


,topic,context,response,prompt,fluency/explanation,fluency/score,text_quality/explanation,text_quality/score
0,Heat Wave,Temperatures reaching 105F. Cooling centers op...,"Heads up, residents! ☀️ Extreme heat is here, ...",Write a social post about Heat Wave. Details: ...,The response is completely free of grammatical...,5.0,"The response is exceptionally clear, coherent,...",5.0
1,Road Closure,Main St closed for parade Saturday 9am-12pm.,"Heads up, residents! 🚧 Main St will be tempora...",Write a social post about Road Closure. Detail...,"The response is grammatically perfect, uses ap...",5.0,"The response is exceptionally clear, coherent,...",5.0
